# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install -q crewai==0.108.0 crewai_tools==0.38.1 langchain_community==0.3.20 langchain_ollama  ollama==0.4.7 python-dotenv
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew, LLM

In [3]:
import os
from dotenv import load_dotenv

load_dotenv('creds.env')  # Adjust path if needed


ollama_llm = LLM(
  	model="ollama/phi4:latest",
  	api_base="http://localhost:11434"
  )


## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm=ollama_llm,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    llm=ollama_llm,
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/skasmani/Downloads/personal/github/CrewAI-Worksatation/crewai-venv/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/skasmani/Downloads/personal/github/C

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [14]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  embedder={
        "provider": "ollama",
        "config": {
            "model": "mxbai-embed-large"
        }
    },
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [15]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 82c6fafe-28b9-4927-a598-f7ba7125c4d8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 66449f6d-504b-4fad-8068-cec2ca82f3ff
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 66449f6d-504b-4fad-8068-cec2ca82f3ff
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


🚀 Crew: crew
└── 📋 Task: 66449f6d-504b-4fad-8068-cec2ca82f3ff
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 66449f6d-504b-4fad-8068-cec2ca82f3ff
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: f15b4efa-6469-4a42-893d-d1aae93a1b19
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 66449f6d-504b-4fad-8068-cec2ca82f3ff
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: f15b4efa-6469-4a42-893d-d1aae93a1b19
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f15b4efa-6469-4a42-893d-d1aae93a1b19                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 82c6fafe-28b9-4927-a598-f7ba7125c4d8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Answer:

Dear Andrew Ng from DeepLearningAI,

Thank you for reaching out with your question about adding memory to your Crew. To add memory to your Crew in CrewAI, you can follow these steps:

1. Create a new Crew: Start by creating a new Crew in CrewAI. A Crew in CrewAI is like an organization of AI agents with specialized roles collaborating to accomplish complex tasks. You can create specialized agents with defined roles, expertise, and goals, such as researchers, analysts, or writers.

2. Assign memory-related tasks: Once you have created your Crew, you can assign memory-related tasks to the AI agents within the Crew. These tasks can involve storing and retrieving information, processing data, or any other memory-related operations.

3. Equip agents with tools: Make sure to equip your AI agents with custom tools and APIs to interact with external services and data sources. This will enable them to effectively work on memory-related tasks and collaborate with other agents in t

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
# Extract the Markdown string from the CrewOutput object
markdown_content = result.raw

# Display the Markdown content
display(Markdown(markdown_content))
# Markdown(result)

Hello,

Thank you for reaching out to DeepLearningAI regarding our CrewAI framework! We're thrilled to provide detailed insights into how Crews and Flows can be utilized to achieve your goals effectively. Here’s a comprehensive breakdown:

**Crews Overview:**
Crews are the backbone of autonomous collaboration within CrewAI. They manage teams of AI agents, each with specific roles and expertise, making them ideal for tasks that require:
- **Autonomous Problem-Solving**: Agents work independently to solve problems.
- **Creative Collaboration**: Multiple agents collaborate creatively on open-ended tasks.
- **Exploratory Tasks**: Engaging in research or content generation where flexibility and innovation are key.

**Key Features of Crews:**
1. **Role-Based Agents**: Customize agent roles based on task requirements, allowing for specialized contributions from each team member.
2. **Flexible Tools**: Equip agents with the necessary tools and APIs to interact seamlessly with external services and data sources.
3. **Intelligent Collaboration**: Facilitate dynamic teamwork where agents share insights and coordinate efforts toward complex goals.
4. **Task Management**: Define workflows that allow tasks to be executed sequentially or in parallel, managing dependencies automatically for efficiency.

**Ideal Use Cases for Crews:**
Crews shine in scenarios like collaborative article generation, conducting exploratory research, or any task where creative thinking and adaptive problem-solving are essential. They provide the autonomy needed for agents to interact dynamically, fostering a rich environment of idea exchange and innovation.

**Flows Overview:**
Flows offer structured workflow orchestration with deterministic execution paths, perfect for tasks requiring precision and control. These components are crucial when:
- **Precision is Key**: Tasks demand specific, controlled sequences.
- **Auditability Needed**: Processes must be transparent and verifiable.
- **Control Over Execution**: Fine-grained management of states and conditional logic ensures reliability.

**Key Features of Flows:**
1. **Event-Driven Orchestration**: Define execution paths that respond to specific events, ensuring tasks are triggered at the right time.
2. **Fine-Grained Control**: Manage intricate details of processes securely and efficiently, maintaining strict oversight.
3. **Native Crew Integration**: Seamlessly combine with Crews to enhance automation while retaining intelligent functionalities.
4. **Deterministic Execution**: Guarantee predictable outcomes through explicit control flow management and robust error handling.

**Ideal Use Cases for Flows:**
Flows are best suited for processes like API orchestration, where precise execution is critical, or any scenario that demands reliable, auditable workflows. They ensure tasks proceed in a controlled manner, integrating with external services seamlessly.

**When to Choose Crews vs. Flows:**
- **Use Crews** when your project needs autonomous decision-making and creative input.
- **Opt for Flows** if your task requires strict control over execution paths and auditability.
- **Combine Both** for hybrid applications where structured processes need elements of autonomy, such as orchestrating a broad process with specific tasks handled by Crews.

CrewAI is designed to balance autonomous operations with structured workflow automation, making it adaptable across diverse applications. Whether you're building your first crew or integrating advanced decision-making capabilities, our framework supports secure and scalable AI agent orchestration tailored for real-world challenges.

We hope this detailed explanation helps! If you have any more questions or need further clarification, feel free to reach out.

Best regards,
[Your Name]
DeepLearningAI Support Team